## 定数定義
$$
V=\{1,\ldots,n\}:アイテムの集合\\
B=\{1,\ldots,n\}:ビンの集合\\
M=\{1,\ldots,m\}:グループの集合\\
w_i:アイテムiの大きさ\\
z_{il}\in\{0,1\}:アイテムiのグループlへの所属を表すバイナリ定数\\
r_l:グループlがビンを1つ使用するのにかかるコスト\\
C:ビンの容量\\
E=\{(s,t) ~|~ w_s+w_t > C~または~sとtは同じビンに割り当てられられない\}:枝の集合\\
G=(V,E):~VとEから成るグラフ\\
$$

## 変数定義
$$
x_{ik}\in\{0,1\}:アイテムiのビンkへの割り当てを表すバイナリ変数\\
y_k\in\{0,1\}:ビンkの使用を表すバイナリ変数\\
b_{kl}\in\{0,1\}:グループlのビンkの使用を表すバイナリ変数\\
$$

## 目的関数
$$
\begin{align}
\text{minimize} & \displaystyle\sum_{k=1}^n y_k + \sum_{k=1}^n \sum_{l=1}^m r_l b_{kl} - \sum_{l=1}^m r_l
\end{align}
$$

## 制約式
$$
\begin{align}
\text{subject to}
\displaystyle\sum_{i=1}^n w_i x_{ik} \leq C y_k&~~~~~(k=1,\dots,n)\\
\displaystyle\sum_{k=1}^n x_{ik} = 1&~~~~~(i=1,\dots,n)\\
x_{sk}+x_{tk} \leq 1&~~~~~((s,t)\in E,~k=1,\dots,n)\\
b_{kl} \geq x_{jk} z_{jl}&~~~~~(k=1,\dots,n,~l=1,\dots,m,~j=1,\dots,n)\\
x_{ik}\in\{0,1\}&~~~~~(i,k=1,\dots,n)\\
y_k\in\{0,1\}&~~~~~(k=1,\dots,n)\\
b_{kl}\in\{0,1\}&~~~~~(k=1,\dots,n,~l=1,\dots,m)\\
\end{align}
$$

# 問題設定

In [66]:
import numpy as np
import random

## 基本設定

In [67]:
n = 6
w = [3, 2, 5, 6, 1, 1]
C = 9
E = [(0,2), (1,2)]
#G = [(0,1,2,3,4,5)]
G = [(0,3), (1,4,5), (2,)]
r = [1]*len(G)
bin_cost = 100

## コンフリクト辞書とノットコンフリクト辞書の作成

In [68]:
N = np.arange(0,n)
E_dict={}
notconf_E_dict={}
flag=E[0][0]
for i in range(len(E)):
    if flag!=E[i][0]:
        flag=E[i][0]
    E_dict[E[i][0]] = E_dict.setdefault(E[i][0],[]) + [E[i][1]]
    E_dict[E[i][1]] = E_dict.setdefault(E[i][1],[]) + [E[i][0]]
    notconf_E_dict[E[i][0]] = list(set(N)-set(E_dict[E[i][0]])-{E[i][0]})
    notconf_E_dict[E[i][1]] = list(set(N)-set(E_dict[E[i][1]])-{E[i][1]})

In [69]:
E_dict

{0: [2], 2: [0, 1], 1: [2]}

### コンフリクトがないノード間に枝があるような集合

In [70]:
notconf_E = []
for i in notconf_E_dict:
    for j in notconf_E_dict[i]:
               notconf_E.append((i,j))

# FFD 

In [71]:
def capacity(w_i,bin_b,w,C): #bin_bの容量がw_iの大きさよりも大きいかを判定
    if (C-sum(w[i] for i in bin_b)) >= w_i:
        return True
    else:
        return False

In [72]:
def conflict(item,bin_b,E_dict): #bin_bに割り当てられているアイテムたちとw_iの間にコンフリクトがあるかを判定
    if item not in E_dict:
        return True
    for i in bin_b:
        if i in E_dict[item]:
                return False
    return True

In [73]:
bin_dict = {}
for i in range(len(N)):
    bin_dict[i] = []
for i in range(len(N)): #アイテムをすべて走査
    for b in range(len(bin_dict)): #インデックスが小さい順にビンを走査
        if capacity(w[i],bin_dict[b],w,C) and conflict(i,bin_dict[b],E_dict):
            bin_dict[b] = bin_dict.setdefault(b,[]) + [i]
            break  

In [74]:
for i in range(len(bin_dict)):
    print(i,bin_dict[i])

0 [0, 1, 4, 5]
1 [2]
2 [3]
3 []
4 []
5 []


In [75]:
bin_counter=0
for i in range(len(bin_dict)):
    if len(bin_dict[i]) > 0:
        bin_counter+=1
print(bin_counter)

3


# 走査するアイテムの順をシャッフルする

In [59]:
random_list = random.shuffle(N)

In [60]:
print(N)

[1 2 5 4 3 0]


## FFDの実行

In [62]:
bin_dict = {}
for i in N:
    bin_dict[i] = []
for i in N: #アイテム
    for b in range(len(bin_dict)): #ビン
        if capacity(w[i],bin_dict[b],w,C) and conflict(i,bin_dict[b],E_dict):
            bin_dict[b] = bin_dict.setdefault(b,[]) + [i]
            break  

In [63]:
for i in range(len(bin_dict)):
    print(i,bin_dict[i])

0 [1, 5, 4, 0]
1 [2]
2 [3]
3 []
4 []
5 []


In [64]:
bin_counter=0
for i in range(len(bin_dict)):
    if len(bin_dict[i]) > 0:
        bin_counter+=1
print(bin_counter)

3
